#### Import Libraries

In [1]:
import numpy as np

#### Define Classes for Building Neural Network Architecture

In [2]:
class Neuron:
    def __init__(self, weights, bias):
        self.weights = weights
        self.bias = bias
        self.output_weights = []
    def forward_propogate_neuron(self, inputs, activation):
        self.activation_obj = get_activation_obj(activation)
        self.inputs = inputs
        self.input_to_activation = np.dot(self.inputs, self.weights) + self.bias
        self.activation_obj.activation(self.input_to_activation)
        self.output = self.activation_obj.output
    def backward_propogate_neuron_last_layer(self, diff_binary_loss, activation, learning_rate):
        self.activation_obj = get_activation_obj(activation)
        self.activation_obj.derivation(self.input_to_activation)
        self.delta = diff_binary_loss * self.activation_obj.output
        self.backward_outputs = np.multiply(self.delta, self.weights)
        self.derivative = np.multiply(self.inputs, self.backward_outputs)
        self.weights = self.weights - learning_rate*self.derivative[0]
    def backward_propogate_neuron_rest_layer(self, backward_input, activation, learning_rate):
        self.activation_obj = get_activation_obj(activation)
        self.activation_obj.derivation(self.input_to_activation)
        self.delta = backward_input * self.activation_obj.output
        self.derivative = np.dot(self.delta, self.inputs)
        self.weights = self.weights - learning_rate*self.derivative
    def update_weights(self, learning_rate):
        self.derivative = np.dot(self.delta, self.inputs)
        self.weights = self.weights - learning_rate*self.derivative

In [3]:
class Layer:
    def __init__(self, n_inputs, n_outputs, activation='linear'):
        self.input_shape = n_inputs
        self.output_shape = n_outputs
        self.activation = activation
        self.neurons = []
        self.deltas = []
    def forward_propogate_layer(self, inputs):
        neuron_count = 1
        layer_outputs = []
        self.inputs = inputs
        for neuron in self.neurons:
            neuron.forward_propogate_neuron(self.inputs, self.activation.activation_name)
            layer_outputs.append(neuron.output)
            neuron_count += 1
        self.outputs = layer_outputs
    def backward_propogate_last_layer(self, diff_binary_loss, learning_rate):
        self.backward_outputs = []
        neuron_count = len(self.neurons)
        for neuron in self.neurons:
            neuron.backward_propogate_neuron_last_layer(diff_binary_loss, self.activation.activation_name, learning_rate)
            self.backward_outputs.append(neuron.backward_outputs)
            neuron_count -= 1
    def backward_propogate_rest_layers(self, prev_layer_outputs, learning_rate):
        self.backward_outputs = [] * len(self.neurons)
        for i in range(len(self.neurons)):
            self.neurons[i].delta = np.array(prev_layer_outputs).T[i] * self.activation.derivation(self.neurons[i].input_to_activation)
            self.deltas.append(self.neurons[i].delta)
            self.neurons[i].update_weights(learning_rate)
        supertmp = []
        for i in range(len(self.neurons)):
            tmp = []
            for j in range(len(self.neurons[i].weights)):
                tmp.extend(self.neurons[i].delta * self.neurons[i].weights[j])
            supertmp.append(tmp)
        self.backward_outputs = np.sum(supertmp, 0, keepdims=True)

In [4]:
class Network:
    def __init__(self, epochs = 1):
        self.network_name = 'My Network'
        self.deltas = []
        self.epochs = epochs
    def initialize_neurons(self):
        if len(self.layers) > 0:
            layer_count = 1
            for layer in self.layers:
                n_inputs = layer.input_shape
                n_neurons = layer.output_shape
                layer_count += 1
                neuron_count = 1
                for i in range(n_neurons):
                    neuron = Neuron(weights = np.random.randn(n_inputs) * 0.1, bias = np.random.randn(1)*0.1)
                    layer.neurons.append(neuron)
                    neuron_count += 1
    def forward_propogate_network(self, inputs):
        if len(self.layers) > 0:
            layer_count = 1
            layer_inputs = inputs
            for layer in self.layers:
                if (layer_count == 1):
                    layer.forward_propogate_layer(layer_inputs)
                else:
                    layer.forward_propogate_layer(np.array(layer_inputs).T)
                layer_count += 1
                layer_inputs = layer.outputs
    def calculate_binary_loss(self, y_prob, y):
        m = y_prob.shape[0]
        y_prob_clipped  = np.clip(y_prob, 1e-7, 1-1e-7) # we are clipping values so that we do not get log(0) kind of situation
        loss = -1/m * (np.dot(np.array(y), np.log(y_prob_clipped)) + np.dot(np.array(1-y), np.log(1-y_prob_clipped)))
        self.loss = np.squeeze(loss)
    def calculate_diff_binary_loss(self, y_prob, y):
        m = y_prob.shape[0]
        self.diff_binary_loss = 1/m * ((1 - np.array(y))/(1 -np.array(y_prob)) - np.array(y)/np.array(y_prob))
    def get_predictions(self, y_prob):
        self.predictions = []
        for i in y_prob:
            if i > 0.5:
                self.predictions.append(1)
            else:
                self.predictions.append(0)
    def get_accuracy(self, y, y_prob):
        self.accuracy = np.mean(self.predictions == y)
    def backward_propogate(self, learning_rate, y_prob, y):
        for i in reversed(range(len(self.layers))):
            if i == len(self.layers) - 1:
                self.calculate_diff_binary_loss(y_prob, y)
                self.layers[i].backward_propogate_last_layer(self.diff_binary_loss, learning_rate)
            else:
                self.layers[i].backward_propogate_rest_layers(self.layers[i+1].backward_outputs, learning_rate)

In [5]:
def get_activation_obj(activation):
        if activation == 'Linear':
            return Activation_Linear()
        elif activation == 'ReLU':
            return Activation_ReLU()
        elif activation == 'Tanh':
            return Activation_Tanh()
        elif activation == 'Sigmoid':
            return Activation_Sigmoid()
        elif activation == 'Softmax':
            return Activation_Softmax()
        else:
            return None

In [6]:
class Activation_ReLU:
    def __init__(self):
        self.activation_name = 'ReLU'
    def activation(self, inputs):
        self.output = np.maximum(0, inputs)
    def derivation(self, outputs):
        if outputs <= 0:
            self.output = 0
        else:
            self.output = outputs
        return self.output

In [7]:
class Activation_Sigmoid:
    def __init__(self):
        self.activation_name = 'Sigmoid'
    def activation(self, inputs):
        self.output = 1/(1+np.exp(-inputs))
    def derivation(self, outputs):
        self.output = 1 - 1/(1+np.exp(-outputs))

In [8]:
class Activation_Softmax:
    def __init__(self):
        self.activation_name = 'Softmax'
    def activation(self, inputs):
        exp_values = np.exp(inputs)
        self.output = exp_values/np.sum(exp_values, axis=0, keepdims=True)
    def derivation(self, outputs):
        pass

In [9]:
class Activation_Tanh:
    def __init__(self):
        self.activation_name = 'Tanh'
    def activation(self, inputs):
        self.output = (np.exp(2*inputs) - 1)/(np.exp(2*inputs) + 1)
    def derivation(self, outputs):
        self.output = 4/(np.exp(-outputs) + np.exp(outputs))**2
        return 4/(np.exp(-outputs) + np.exp(outputs))**2

In [10]:
class Activation_Linear:
    def __init__(self):
        self.activation_name = 'Linear'
    def activation(self, inputs):
        self.output = inputs
    def derivation(self, outputs):
        self.output = 0

#### Initialize Neural Network Architecture

In [11]:
import pandas as pd

In [12]:
df = pd.read_csv('irisTestData.csv')

In [13]:
np.random.seed(42)

# X, y = create_data(samples = 1, classes = 1, n_features = 5)
layers = []
layers.append(Layer(n_inputs = len(df.columns)-1, n_outputs = 6, activation = Activation_ReLU()))
layers.append(Layer(n_inputs = 6, n_outputs = 6, activation = Activation_Tanh()))
layers.append(Layer(n_inputs = 6, n_outputs = 4, activation = Activation_Tanh()))
layers.append(Layer(n_inputs = 4, n_outputs = 5, activation = Activation_Tanh()))
layers.append(Layer(n_inputs = 5, n_outputs = 4, activation = Activation_Tanh()))
layers.append(Layer(n_inputs = 4, n_outputs = 4, activation = Activation_Tanh()))
layers.append(Layer(n_inputs = 4, n_outputs = 5, activation = Activation_Tanh()))
layers.append(Layer(n_inputs = 5, n_outputs = 1, activation = Activation_Sigmoid()))

network = Network(epochs = 10)
network.layers = layers
network.initialize_neurons()

In [15]:
for i in range(network.epochs):
#     print('Epoch #{0}'.format(i+1))
    for index, row in df.iterrows():
        lst_prob = []
        print('Epoch #{1} and Sample # {0}'.format(index + 1, i + 1))
        print('-----------------------------------------------------------')
        X = row[:-1]
        y = row[-1]
        network.forward_propogate_network(np.array(X).reshape(1,len(X)))
        print('Forward Propogation Completed Successfully!')
        y_prob = network.layers[-1].neurons[0].output
        lst_prob.append(y_prob)
        network.calculate_binary_loss(y_prob, y)
        print('Loss = ', network.loss)
        network.backward_propogate(0.1, y_prob, y)
        print('Backward Propogation Completed Successfully!')
        network.get_predictions(y_prob)
    network.get_accuracy(np.array(df['y']), lst_prob)
    print('Accuracy = ', network.accuracy)
    print('===========================================================')

Epoch #1
Accuracy =  0.6666666666666666
Epoch #2
Accuracy =  0.6666666666666666
Epoch #3
Accuracy =  0.6666666666666666
Epoch #4
Accuracy =  0.6666666666666666
Epoch #5
Accuracy =  0.6666666666666666
Epoch #6
Accuracy =  0.6666666666666666
Epoch #7
Accuracy =  0.6666666666666666
Epoch #8
Accuracy =  0.6666666666666666
Epoch #9
Accuracy =  0.6666666666666666
Epoch #10
Accuracy =  0.6666666666666666
